In [2]:
import pandas as pd
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列

In [3]:
import pandas as pd

# 读入 CSV 文件
# df = pd.read_csv('evaluate_algorihmic.csv')
df = pd.read_csv('evaluate_seg_trans#.csv')

df = df.drop(columns=['pbid'])
df = df[df['ARG'] <= 100000]
# df = df[df['classcial'] <= 500]
# df = df[df['in_constraints_probs'] >= 99]
grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
    'iteration_count':'mean',
    'classcial':'mean',
    'run_times':'mean',
    "ARG": 'mean',
    'in_constraints_probs': 'mean',
    'best_solution_probs': 
    'mean',
})

## 分组并把组作为索引
pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=["ARG", 'best_solution_probs', 'classcial', 'run_times','iteration_count'])
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

# method_order = ['ChocoSolver', 'QtoSimplifySolver', 'QtoSimplifyDiscardSolver', 'QtoSimplifyDiscardSegmentedSolver']
# method_order = ['HeaSolver', 'PenaltySolver', 'ChocoSolver', 'QtoSolver ', 'QtoSimplifySolver', 'QtoSimplifyDiscardSolver', 'QtoSimplifyDiscardSegmentedSolver']
# method_order = ['ChocoSolver', 'QtoSolver', 'QtoSimplifySolver', 'QtoSimplifyDiscardSolver']
# method_order = ['ChocoSolver']
method_order = ['QtoSimplifyDiscardSegmentedSolver']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['classcial', "ARG",'best_solution_probs','run_times','iteration_count'], method_order]))

pivot_df

,,,classcial,ARG,best_solution_probs,run_times,iteration_count
,,,QtoSimplifyDiscardSegmentedSolver,QtoSimplifyDiscardSegmentedSolver,QtoSimplifyDiscardSegmentedSolver,QtoSimplifyDiscardSegmentedSolver,QtoSimplifyDiscardSegmentedSolver
pkid,variables,constraints,,,,,
0,6,3,0.531756,0.001441,99.749023,152391.68,191.87
1,15,8,76.457449,0.104129,79.954102,9789194.24,301.00
2,21,12,455.766491,0.338501,34.457031,19134545.92,301.00


In [4]:
# # 假设“variables”和“constraints”列作为多重索引列的一部分
# # 使用 .loc 选择多重索引列中含“variables”和“constraints”相关的列
# selected_columns = pivot_df.loc[:, pivot_df.columns.get_level_values(0).isin(['variables', 'constraints'])]

# selected_columns


In [5]:
# 选取 'depth' 下的所有列，并剔除 'QtoSolver ' 和 'QtoSimplifySolver'
df_1 = pivot_df['ARG'].drop(columns=['QtoSolver ', 'QtoSimplifySolver'], errors='ignore')

# 选取 'culled_depth_per_param' 下的 'QtoSimplifyDiscardSolver' 列
df_2 = pivot_df['classcial'].drop(columns=['QtoSolver ', 'QtoSimplifySolver'], errors='ignore')

# 合并处理后的 'depth' 列与 'QtoSimplifyDiscardSolver' 列，并指定后缀避免冲突
culled_depth_df = df_1.join(df_2, lsuffix='_1', rsuffix='_2')

# 按行遍历输出值，每个值用 & 分隔
for row in culled_depth_df.itertuples(index=False):
    formatted_row = [f"{round(x, 3):6.03f}" for x in row]
    print(" & ".join(formatted_row))


 0.001 &  0.532
 0.104 & 76.457
 0.339 & 455.766


In [6]:
# # 从 pivot_df 中提取 variables 和 constraints 列
# variables_constraints_df = pivot_df.index.to_frame(index=False)[['variables', 'constraints']]
# variables_constraints_df

In [7]:
# 提取所需列
best_solution_probs_Qto_discard = pivot_df['best_solution_probs']['QtoSimplifyDiscardSolver']
best_solution_probs_choco = pivot_df['best_solution_probs']['ChocoSolver']
arg_Qto_discard = pivot_df['ARG']['QtoSimplifyDiscardSolver']
arg_choco = pivot_df['ARG']['ChocoSolver']

# 方法1：每行相除取平均
# best_solution_probs 下 QtoSimplifyDiscardSolver / ChocoSolver 的倍数
row_wise_ratio_best_solution_probs = (best_solution_probs_Qto_discard / best_solution_probs_choco).mean()

# ARG 下 ChocoSolver / QtoSimplifyDiscardSolver 的倍数
row_wise_ratio_arg = (arg_choco / arg_Qto_discard).mean()

# 方法2：整列取平均后相除
# best_solution_probs 下 QtoSimplifyDiscardSolver / ChocoSolver 的整列平均的倍数
col_avg_ratio_best_solution_probs = best_solution_probs_Qto_discard.mean() / best_solution_probs_choco.mean()

# ARG 下 ChocoSolver / QtoSimplifyDiscardSolver 的整列平均的倍数
col_avg_ratio_arg = arg_choco.mean() / arg_Qto_discard.mean()

# 输出结果
print(f"行平均 - QtoSimplifyDiscardSolver / ChocoSolver (best_solution_probs): {row_wise_ratio_best_solution_probs:.2f}")
print(f"行平均 - ChocoSolver / QtoSimplifyDiscardSolver (ARG): {row_wise_ratio_arg:.2f}")
print(f"列平均 - QtoSimplifyDiscardSolver / ChocoSolver (best_solution_probs): {col_avg_ratio_best_solution_probs:.2f}")
print(f"列平均 - ChocoSolver / QtoSimplifyDiscardSolver (ARG): {col_avg_ratio_arg:.2f}")


KeyError: 'QtoSimplifyDiscardSolver'

In [ ]:

# # 读入 CSV 文件
# df = pd.read_csv('evaluate_TSP.csv')

# df = df.drop(columns=['pbid'])

# grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
#     "ARG": 'mean',
#     'in_constraints_probs': 'mean',
#     'best_solution_probs': 'mean',
#     'iteration_count':'mean'
# })

# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

# method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df

In [ ]:
# import pandas as pd

# # 读入 CSV 文件
# df = pd.read_csv('eva_SCP.csv')

# df = df.drop(columns=['pbid'])

# grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
#     "ARG": 'mean',
#     'in_constraints_probs': 'mean',
#     'best_solution_probs': 'mean',
#     'iteration_count':'mean'
# })

# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

# method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df

In [ ]:
# 定义格式化函数
import numpy as np
def format_value(val):
    if np.isnan(val):  # 如果是NaN值则返回空字符
        return "NaN"
    elif val < 100:  # 如果值小于100，保留3位有效数字
        return f"{val:.2f}"
    else:  # 如果值大于等于100，不保留小数点
        return f"{val:.0f}"

# 按行遍历输出，每个值用 & 分隔
for row in pivot_df.itertuples(index=False):
    formatted_row = [format_value(x) for x in row]
    print(" & ".join(formatted_row))

NaN & NaN & NaN & NaN & NaN & NaN & NaN & NaN & NaN & NaN & NaN & NaN & NaN & NaN & NaN


In [ ]:
# import pandas as pd

# # 读入 CSV 文件
# df = pd.read_csv('eva_old.csv')

# df = df.drop(columns=['pbid'])

# grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
#     "ARG": 'mean',
#     'in_constraints_probs': 'mean',
#     'best_solution_probs': 'mean',
#     'iteration_count':'mean'
# })
# grouped_df
# # 假设grouped_df是一个Pandas DataFrame
# grouped_df = grouped_df[((grouped_df['layers'] == 1) & (grouped_df['method'] == 'commute')) | ((grouped_df['layers'] == 7) & (grouped_df['method'] != 'commute'))]
# grouped_df = grouped_df.drop(columns=['layers'])
# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])




# method_order = ['penalty', 'cyclic', 'HEA', 'commute']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df

In [ ]:
pivot_df['ARG']

,,,ChocoSolver,QtoSimplifySolver,QtoSimplifyDiscardSolver
pkid,variables,constraints,,,
0,6,3,NaN,NaN,NaN


In [ ]:
# import pandas as pd

# # 读入 CSV 文件
# df = pd.read_csv('eval_eliminate copy 2.csv')

# df = df.drop(columns=['pbid','layers', 'variables', 'constraints'])

# gr    ouped_df = df.groupby(['pkid',  'method'], as_index=False).agg({
#     "successrate": 'mean',
# })
# grouped_df
# # 假设grouped_df是一个Pandas DataFrame
# # grouped_df = grouped_df[((grouped_df['layers'] == 1) & (grouped_df['method'] == 'commute')) | ((grouped_df['layers'] == 7) & (grouped_df['method'] != 'commute'))]
# # grouped_df = grouped_df.drop(columns=['layers'])
# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])




# method_order = ['penalty', 'cyclic', 'HEA', 'commute']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df